<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install
!pip install pdfminer.six
!pip install spacy

     |████████████████████████████████| 5.6MB 1.4MB/s 
     |████████████████████████████████| 13.7MB 308kB/s 


In [2]:
!python -m spacy download fr_core_news_sm
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 14.7MB 1.0MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=84fa149e1aa7b86f577640322e17e5261fd1ddd27cebcf0ea8e2931b1c857d93
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzygmjxj/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 88.6MB 1.1MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=6544172325418d6a51439d068e81b075787f87bda6175b51fcc94a5c1b765d8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-btcj_7jx/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [4]:
from google.colab import files

uploaded = files.upload()


Saving dataset.json to dataset.json


In [5]:
ls

dataset.json  sample_data/


In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [0]:
pdfs = []
for f in uploaded.keys():
  try:
    text=extract_text_from_pdf(f)
    if "Déclaration de franchissement de seuils" in text:
      pdfs.append({
          "file": f,
          "text": text,
      })
  except:
    pass

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
import json
with open('dataset.json') as f:
  pdfs = json.load(f)

In [0]:
import IPython
from google.colab import output

class AnnotWidget():
  def __init__(self, to_annot):
    self.to_annot = to_annot
    self.cur = 0
    self.colors_labels = [
        ("rgb(255, 0, 0)", "ISSUER"),
        ("rgb(255, 0, 255)", "FIRM"),
        ("rgb(0, 255, 255)", "NUMBER"),
        ("rgb(160, 160, 255)", "PERCENT"),
        ("rgb(192, 192, 192)", "TYPE", ),
        ("rgb(255, 255, 0)", "DIRECTION"),
        ("rgb(0, 255, 0)", "DATE")
      ]
    self.controls = ""
    for c, l in self.colors_labels:
      self.controls += '''<button onclick='document.execCommand("backColor", false, "'''+c+'''")'>'''+l+'''</button>'''

  def text_entities_to_html(self):
    if 'entities' not in self.to_annot[self.cur] or len(self.to_annot[self.cur]['entities']) == 0:
      return self.to_annot[self.cur]["text"]
    res = ""
    offset = 0
    text = self.to_annot[self.cur]["text"]
    for ent_s, ent_e, label in self.to_annot[self.cur]['entities']:
      res += text[offset:ent_s]
      res += '''<span style="background-color: '''+ self.get_color(label) + '''">'''
      res += text[ent_s:ent_e]
      res += "</span>"
      offset = ent_e
    res += text[offset:]
    return res

  def run(self):
    display(IPython.display.HTML('''
    <p>'''+str(self.cur) + "/" + str(len(self.to_annot)) + '''</p>
    <div id="ed" contenteditable="true">'''+self.text_entities_to_html()+'''</div>
    <fieldset>
    '''+ self.controls +'''
      <button onclick="document.execCommand('removeFormat', false, null)">remove</button>
      <button id='prev'>prev</button>
      <button id='next'>next</button>
    </fieldset>
    <script>
      function change(dir) {
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, rgb])
          }
          offset = next_offset;
        }
        next_text = google.colab.kernel.invokeFunction('notebook.SaveEntities', [res, dir]);
        document.querySelector('#ed').innerHTML = next_text;
      };
      document.querySelector('#prev').onclick = () => {
        change(-1)
      };
      document.querySelector('#next').onclick = () => {
        change(1)
      }
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)

  def get_color(self, label):
    for c, l in self.colors_labels:
        if l == label:
          return c

  def get_label(self, color):
    for c, l in self.colors_labels:
        if c == color:
          return l

  def save_entities(self, res, inc=1):
    self.to_annot[self.cur]['entities'] = []
    for ent_s, ent_e, color in res:
      self.to_annot[self.cur]['entities'].append((ent_s, ent_e, self.get_label(color)))
    output.clear()
    if (self.cur < len(self.to_annot) and inc == 1) or (self.cur > 0 and inc == -1):
      self.cur += inc
    self.run()


w = AnnotWidget(pdfs) 

In [20]:
w.run()

In [0]:
import json
with open("dataset.json", 'w+') as f:
  json.dump(w.to_annot, f)

In [0]:
train_data = []
for x in w.to_annot[:13]:
  entities = [tuple(ent) for ent in  x["entities"]]
  train_data.append((x['text'], {'entities' : entities}))
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE']
test_data = train_data[:2]
train_data = train_data[2:]

In [29]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=30
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
    sizes = compounding(1.0, 4.0, 1.001)
    # batch up the examples using spaCy's minibatch
    for itn in range(n_iter):
        random.shuffle(train_data)
        batches = minibatch(train_data, size=sizes)
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        print("Losses", losses)

Losses {'ner': 3784.534862152256}
Losses {'ner': 3148.0505520086735}
Losses {'ner': 2964.5361043633893}
Losses {'ner': 2904.3241489715874}
Losses {'ner': 2848.446144092828}
Losses {'ner': 2823.5537864845246}
Losses {'ner': 2808.8471510335803}
Losses {'ner': 2824.875702910591}
Losses {'ner': 2778.7116724615917}
Losses {'ner': 2740.9250082997605}
Losses {'ner': 2774.6721063230652}
Losses {'ner': 2730.4985087886453}
Losses {'ner': 2683.032482760027}
Losses {'ner': 2656.437393248081}
Losses {'ner': 2675.6870335098356}
Losses {'ner': 2725.9327673777007}
Losses {'ner': 2692.891166128218}
Losses {'ner': 2693.7453514635563}
Losses {'ner': 2685.9609784706845}
Losses {'ner': 2652.357140702894}
Losses {'ner': 2684.993739336729}
Losses {'ner': 2692.2840438796266}
Losses {'ner': 2587.298816113267}
Losses {'ner': 2667.1742777680047}
Losses {'ner': 2592.0304921641946}
Losses {'ner': 2675.895578200929}
Losses {'ner': 2699.140206466429}
Losses {'ner': 2581.3309933543205}
Losses {'ner': 2586.33892324566

In [30]:
doc = nlp(train_data[0][0])
for ent in doc.ents:
    print(ent.label_, ent.text)

DATE 20 février 2020
ISSUER EUROPCAR MOBILITY GROUP
DATE 20 février 2020
FIRM Morgan Stanley Corp.
FIRM Corporation Trust Centre, 1209 Orange Street, Wilmington, Delaware DE 19801, Etats
DATE 14 février 2020
PERCENT 5%
TYPE capital
TYPE droits de vote
ISSUER EUROPCAR MOBILITY GROUP
FIRM Morgan Stanley & Co.
FIRM Morgan Stanley & Co.
FIRM Morgan Stanley Capital Services LLC et Morgan Stanley France S.A., 13 372 152 actions EUROPCAR MOBILITY GROUP représentant autant de droits de vote, soit
PERCENT 8,16%
TYPE capital
PERCENT 8,13%
TYPE droits de vote
FIRM Morgan Stanley & Co.
NUMBER 12 825 285
NUMBER 13 372 152
ISSUER EUROPCAR MOBILITY GROUP
FIRM Morgan Stanley & Co.
FIRM Morgan Stanley & Co.
NUMBER 39 659
TYPE actions
ISSUER EUROPCAR MOBILITY GROUP
ISSUER EUROPCAR MOBILITY GROUP
FIRM Morgan Stanley & Co.
NUMBER 218 329
TYPE actions
ISSUER EUROPCAR MOBILITY GROUP
ISSUER EUROPCAR MOBILITY GROUP
NUMBER 163 884 278
TYPE actions
NUMBER 164 457 453
TYPE droits de vote
FIRM Morgan Stanley Capi

In [0]:
!pip install tabula-py

     |████████████████████████████████| 10.4MB 3.3MB/s 


In [0]:
from tabula import read_pdf

list_pdf = list(pdfs.keys())
df = read_pdf(list_pdf[0], pages='all')

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [0]:
import pandas as pd
for d in df:
  if not d.empty:
    print(d)
    table_str = ""
    for c in d.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in d.iterrows():
      for c in d.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    print(table_str)

                          Unnamed: 0  Unnamed: 1  ... % droits de vot Unnamed: 10
0        Merrill Lynch International  14 101 661  ...             NaN         NaN
1          BofA Securities Europe SA     276 021  ...             NaN         NaN
2              BofA Securities, Inc.         213  ...             NaN         NaN
3  Total Bank of America Corporation  14 377 895  ...             NaN         NaN

[4 rows x 15 columns]
Actions % capita Droits de vote % droits de vot Merrill Lynch International 14 101 661 5,49 14 101 661 5,47 BofA Securities Europe SA 276 021 0,11 276 021 0,11 BofA Securities, Inc. 213 ns 213 ns Total Bank of America Corporation 14 377 895 5,59 14 377 895 5,58 


In [0]:
import json
with open("texts.json", "w+") as f:
  json.dump(pdfs, f)


In [0]:
with open("texts.txt", "w+") as f:
  f.write(pdfs[list_pdf[0]]['text'])

In [24]:
import spacy
import fr_core_news_sm
import fr_core_news_md

#nlp = fr_core_news_sm.load()
#nlp = fr_core_news_md.load()

doc = nlp(train_data[0][0])
for sent in doc.sents:
    print(sent)



 220C0490-CH0308403085-FS0132             220C0490 CH0308403085-FS0132 6 février 2020   
Déclaration de franchissement de seuils (article L. 233
-7 du code de commerce)    GENEURO
SA  (Euronext Paris)    Par courrier reçu le 5 février 2020, complété par un courrier reçu le 6 février, la société Invesco Ltd.
(Invesco Head Quarters, Two Peachtree Pointe 1555 Peachtree St, Suite 1800 Atlanta, Georgie, Etats
-Unis), agissant pour le compte du fonds Invesco Advisers, Inc.
dont elle assure la gestion, a déclaré avoir franchi en hausse, le 4 février 2020, les seuils 5% du capital et des droits de vote de la société GENEURO SA et détenir, pour le compte dudit fonds, 1 661 017 actions GENEURO SA représentant autant de droits de vote, soit 8,07% du capital et des droits de vote de cette société1.  
Ce franchissement de seuils résulte de la souscription à l’augmentation de capital de la société GENEURO SA2.  
________                                                            1 Sur la base d’un c

In [0]:
for i in range(len(train_data[0][1]['entities'])):
  s, e, _ = train_data[0][1]['entities'][i]
  print(train_data[0][0][s:e])


21 février 2020
ALTRAN TECHNOLOGIES
20 février 2020
21 février
Bank of America Corporation
en hausse
17 février 2020
5%
capital
droits de vote
ALTRAN TECHNOLOGIES
14 377 895
actions
ALTRAN TECHNOLOGIES
5,59%
capital
5,58%
droits de vote
ALTRAN TECHNOLOGIES
ALTRAN TECHNOLOGIES
Merrill Lynch International
en hausse
Merrill Lynch International
13 784 025
actions
ALTRAN TECHNOLOGIES
Merrill Lynch International
BofA Securities Europe SA
274 482
actions
ALTRAN TECHNOLOGIES
BofA Securities Europe SA
Merrill Lynch International
17 749
actions
ALTRAN TECHNOLOGIES


In [0]:
from spacy import displacy

In [31]:
displacy.render(doc, style='ent', jupyter=True)

In [0]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)
 

220C0170-FR0012407096-OP028-AS12-RO02            220C0170 FR0012407096-OP028-AS12-RO02 14 janvier 2020    NP 220C0170-FR0012407096-OP028-AS12-RO02
- Mise en œuvre du retrait obligatoire visant les actions de la société.   NP Mise
- Maintien de la suspension de la cotation des actions de la société.     NP Maintien
Euronext Paris NP Euronext
Le 13 janvier 2020 NP janvier
l’Autorité des marchés financiers NP Autorité
d’achat simplifiée NP achat
les actions ITS GROUP NP actions
la société ITS Participations NP société
actions ITS GROUP NP actions
de droits de vote NP droits
des droits de vote de cette société2 NP droits
cf NP cf
D&I 220C0149 en date du 13 janvier 2020).   NP D&I
Les actions ITS GROUP non présentées à l’offre par les actionnaires minoritaires NP actions
de 366 396 actions NP actions
483 droits de vote, NP droits
soit 4,65% du capital et 4,92% des droits de vote de cette société2 NP %
2- Par courrier NP 2-
de la société NP société
ITS Participations NP Participations
l’Auto